<a href="https://colab.research.google.com/github/atrbyg24/kilter-board/blob/main/Kilterboard_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from string import digits
from PIL import Image,ImageOps
import time

In [ ]:
def scrapeClimbs(angle,minAscents,minGrade,maxGrade):
    #angle = multiples of 5 between 0 and 70
    #minAscents >=1
    #minGrade <= maxGrade 4a/V0 = 10, 4b/V0 = 11,..., 7c/V9 = 26, 7c+/V10 = 27
    driver = webdriver.Safari()
    driver.maximize_window()
    driver.get(f"https://climbdex.fly.dev/results?name=&angle={angle}&minAscents={minAscents}&minGrade={minGrade}&maxGrade={maxGrade}&sortBy=ascents&sortOrder=desc&minRating=1.0&gradeAccuracy=1&settername=&holds=&mirroredHolds=&board=kilter&layout=1&size=7&set=1&set=20&roleMatch=strict")
    time.sleep(3)

    try:
        climb_results = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"header-results-count")))
        num_climbs = int(''.join(c for c in climb_results.text if c in digits))
        first_climb_button = WebDriverWait(driver,10.).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#div-results-list > button:nth-child(1)")))
        first_climb_button.click()
        climb_count = 1

        for i in range(num_climbs):
            climb_name = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"header-climb-name")))
            climb_stats = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"paragraph-climb-stats")))
            climb_grade = climb_stats.text.split()[0]
            climb_fontgrade = climb_grade.split('/')[0]
            climb_vgrade = climb_grade.split('/')[1]
            try:
                driver.find_element(By.ID,"paragraph-climb-description")
                climb_description = driver.find_element(By.ID,"paragraph-climb-description")
                outlier = False
                if 'none' not in climb_description.get_attribute('class'):
                    climb_text = climb_description.text.lower()
                    if "match" in climb_text or "campus" in climb_text:
                        outlier = True
                if not outlier:
                    climb_box = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#svg-climb")))
                    climb_box.screenshot(rf'drive/MyDrive/Data/{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png')
                    with Image.open(rf'drive/MyDrive/Data{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png') as im:
                        width,height = im.size
                        im_cropped = im.crop(((0, 158, width, height)))
                        im_cropped.save(rf'drive/MyDrive/Data/{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png')
                        im_resized = im_cropped.resize((258, 283), Image.ANTIALIAS)
                        im_resized.save(rf'drive/MyDrive/Data/{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png',quality=95)
                if outlier:
                    climb_box = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#svg-climb")))
                    climb_box.screenshot(rf'drive/MyDrive/Data/Outliers/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png')
                    with Image.open(rf'drive/MyDrive/Data/Outliers/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png') as im:
                        width,height = im.size
                        im_cropped = im.crop(((0, 158, width, height)))
                        im_cropped.save(rf'drive/MyDrive/Data/Outliers/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png')
                        im_resized = im_cropped.resize((258, 283), Image.ANTIALIAS)
                        im_resized.save(rf'drive/MyDrive/Data/Outliers/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png',quality=95)
                climb_count+=1
                right_arrow = WebDriverWait(driver,10.).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#button-next")))
                right_arrow.click()
                time.sleep(2)

            except:
                climb_box = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#svg-climb")))
                climb_box.screenshot(rf'drive/MyDrive/Data/{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png')
                with Image.open(rf'drive/MyDrive/Data/{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png') as im:
                    width,height = im.size
                    im_cropped = im.crop(((0, 158, width, height)))
                    im_cropped.save(rf'drive/MyDrive/Data/{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png')
                    im_resized = im_cropped.resize((258, 283), Image.ANTIALIAS)
                    im_resized.save(rf'drive/MyDrive/Data/{climb_vgrade}/{climb_fontgrade}-{climb_vgrade}-{climb_count}.png',quality=95)
                climb_count+=1
                right_arrow = WebDriverWait(driver,10.).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#button-next")))
                right_arrow.click()
                time.sleep(2)


    except:
            driver.quit()
    driver.quit()


In [1]:
import os
import pandas as pd

# Get the root directory of the dataset
root_dir = 'drive/MyDrive/Data'

# Get a list of all subdirectories (folders) in the root directory
subdirectories = os.listdir(root_dir)

# Create an empty list to store the image paths and labels
image_paths = []
labels = []

labels_map = {
    "V1": 0,
    "V2": 1,
    "V3": 2,
    "V4": 3,
    "V5": 4,
    "V6": 5,
    "V7": 6,
    "V8": 7,
}

# Iterate through each subdirectory
for subdirectory in subdirectories:
    # Get the path of the current subdirectory
    subdirectory_path = os.path.join(root_dir, subdirectory)

    # Get a list of all files in the current subdirectory
    if os.path.isdir(subdirectory_path):
      files = os.listdir(subdirectory_path)

    # Iterate through each file in the current subdirectory
    for file in files:
        # Get the path of the current file
        file_path = os.path.join(subdirectory_path, file)

        # Check if the current file is an image
        if file.endswith('.png'):
            # Add the image path and label to the lists
            image_paths.append(file_path)
            labels.append(labels_map[subdirectory])

# Create a Pandas DataFrame from the lists
df = pd.DataFrame({
    'images': image_paths,
    'labels': labels
})
# Save the DataFrame to a CSV file
df.to_csv('kilterboard.csv', index=False)

In [2]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_image
import os
import pandas as pd

In [3]:
#from skimage import io, transform
from torchvision.io import read_image, ImageReadMode
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision import transforms, utils
from torchvision.transforms import ToTensor
from torchvision.transforms import v2


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()


class KilterboardDataset(Dataset):
    #Kilterboard dataset

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.kilterboard_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.kilterboard_frame)

    def __getitem__(self, idx):
        img_path = self.kilterboard_frame.iloc[idx, 0]
        image = read_image(img_path,mode=ImageReadMode.RGB)
        image = image.type(torch.FloatTensor)
        label = self.kilterboard_frame.iloc[idx, 1]
        return image, label

class ApplyTransform(Dataset):
    """
    Apply transformations to a Dataset

    Arguments:
        dataset (Dataset): A Dataset that returns (sample, target)
        transform (callable, optional): A function/transform to be applied on the sample
    """
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample, target = self.dataset[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

train_transform = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
])


In [4]:
kilterboard_dataset = KilterboardDataset(csv_file='drive/MyDrive/kilterboard.csv',
                                    root_dir='drive/MyDrive/Data')
generator1 = torch.Generator().manual_seed(17)
train_dataset, test_dataset = torch.utils.data.random_split(kilterboard_dataset, [0.8, 0.2],generator = generator1)
transformed_train = ApplyTransform(train_dataset,train_transform)

batch_size = 64

train_dataloader = DataLoader(transformed_train, batch_size=batch_size,shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,shuffle = False)


In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class ConvNeuralNet(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(3, 8, 3)
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(8,16,5)
      self.conv3 = nn.Conv2d(16,32,5)
      self.fc1 = nn.Linear(29696, 1024)
      self.fc2 = nn.Linear(1024, 128)
      self.fc3 = nn.Linear(128,8)

    def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = self.pool(F.relu(self.conv3(x)))
      x = torch.flatten(x, 1) # flatten all dimensions except batch
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)

      return x

model = ConvNeuralNet().to(device)
print(model)

Using cuda device
ConvNeuralNet(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=29696, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=8, bias=True)
)


In [ ]:
from collections import Counter

train_size = len(train_dataset)
train_classes = [label for _, label in train_dataset]
freq = Counter(train_classes)
weights = []
for i in range(8):
  weights.append(train_size/freq[i])
weights = torch.tensor(weights)
print(weights)

In [ ]:
weights = [17.7629, 21.5375,  6.5680,  5.7981,  4.9488,  6.6783,  8.5367,  9.6527]
weights = torch.tensor(weights)
loss_fn = nn.CrossEntropyLoss(weight = weights.to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "kilter_model.pth")
print("Saved PyTorch Model State to kilter_model.pth")

In [ ]:
model = ConvNeuralNet().to(device)
model.load_state_dict(torch.load("kilter_model.pth"))

In [ ]:
classes = ('V1', 'V2', 'V3', 'V4',
           'V5', 'V6', 'V7', 'V8')

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')